In [4]:
import torch
from PIL import Image
import requests
from lavis.models import load_model_and_preprocess

In [5]:
from lavis.datasets.builders import load_dataset
msvd_dataset = load_dataset("msvd_qa")

print(msvd_dataset.keys())
print(len(msvd_dataset["train"]))
print(msvd_dataset["train"][0])



Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_train.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_val.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_test.json
Using downloaded and verified file: /home/ommos92/.cache/lavis/msvd/annotations/qa_ans2label.json
dict_keys(['train', 'val', 'test'])
30933
{'video': '/home/ommos92/.cache/lavis/msvd/videos/-4wsuPCjDBc_5_15.avi', 'text_input': 'what is chewing on a nut?', 'answers': 35, 'question_id': '0000000', 'instance_id': '0'}


In [18]:
#Write a function to convert the .avi into a set of images. Then we can randomly 
# sample from this set of images.
import cv2
from PIL import Image
import random

def convert_avi(data):
    # Open the video file
    cap = cv2.VideoCapture(data['video'])

    images = []
    while(cap.isOpened()):
        # Read the video frame by frame
        ret, frame = cap.read()
        if ret:
            # Convert the frame to RGB (OpenCV uses BGR by default)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            resized_frame = cv2.resize(frame, (224,224))
            # Convert the frame to a PIL Image and append to the list
            images.append(Image.fromarray(resized_frame))

        else:
            break

    # Close the video file
    cap.release()

    # Return the list of PIL Images
    return images

def random_sample(images, num_samples):
    #Randomly select 'num_samples' images from the list
    sampled_images = random.sample(images, num_samples)
    return sampled_images


images = convert_avi(msvd_dataset["train"][0])
print(len(images))

# Show the shope of the image
print(images[0].size)

300
(224, 224)


In [14]:

# setup device to use
device = torch.device("cuda:1") if torch.cuda.is_available() else "cpu"

In [8]:
#Load the model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip2_opt", model_type="pretrain_opt6.7b", is_eval=True, device=device
)


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.63s/it]


In [29]:
# from lavis.tasks.vqa import VQATask

# # Create a VQATask instance
# vqa_task = VQATask(
#     num_beams=3,
#     max_len=10,
#     min_len=1,
#     evaluate=True,
#     num_ans_candidates=128,
#     inference_method="rank",
#     prompt=""
# )

# # Generate predictions
# predictions = vqa_task.valid_step(model, samples=msvd_dataset["test"])


TypeError: list indices must be integers or slices, not str

In [39]:
from lavis.common.vqa_tools.vqa_eval import VQAEval


# Initialize a list to store the accuracy for each video
accuracies = []

# Loop over the videos in the dataset
#for video in msvd_dataset["test"]:
for i, video in enumerate(msvd_dataset["test"]):
    # Sample a random frame from the video
    frame = random_sample(convert_avi(video), 1)
    image = vis_processors["eval"](frame[0]).unsqueeze(0).to(device)
    
    # Get the annotation for the video
    annotation = msvd_dataset['test'].annotation[i]

    # Perform VQA on the frame
    predicted_answer = model.generate(
        {"image" : image, "question" : annotation['question']}
        )
    
    # Get the ground truth from the annotation
    ground_truth_answer = annotation['answer']
    ground_truth_question = annotation['question']
    
    
    # Ensure the predicted question matches the ground truth question
    #assert question == ground_truth_question, "Predicted question does not match ground truth question"
    
    # Create an instance of VQAEval
    vqa_eval = VQAEval(ground_truth_answer, predicted_answer[0], n=2)
    
    # Compute the accuracy
    vqa_eval.evaluate()
    accuracy = vqa_eval.accuracy["overall"]
    
    # Add the accuracy to the list
    accuracies.append(accuracy)
    
    # Print the accuracy for this video
    print(accuracy)

# After the loop, you can do something with the list of accuracies, like compute the average accuracy
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average accuracy: {average_accuracy}")

AttributeError: 'str' object has no attribute 'getQuesIds'